In [ ]:
!pip install modin --user
!pip install "modin[ray]"

In [ ]:
!pip install --upgrade pandas==1.3.0 --user

In [28]:
import numpy as np
import pandas as pd
import os
import glob

from plotly.subplots import make_subplots
import plotly.graph_objects as go
from plotly.offline import plot

import warnings
warnings.filterwarnings("ignore")

In [4]:
path = "./engagement_data"
files = glob.glob(path + "/*.csv")
data = []
for file in files:
    df_raw = pd.read_csv(file)
    df_raw["filename"] = os.path.basename(file)
    data.append(df_raw)

df_engagement_data = pd.concat(data,ignore_index= True)
districts_info_df  = pd.read_csv("districts_info.csv")
products_info_df   = pd.read_csv("products_info.csv")

In [5]:
def missing_df(df):
    missing_values = df.isnull().sum() / len(df) * 100
    df_ = pd.DataFrame( missing_values, columns=["value"] ).reset_index()
    return df_
    
missing_districts_info     = missing_df(districts_info_df)
missing_products_info      = missing_df(products_info_df)
missing_df_engagement_data = missing_df(df_engagement_data)

In [6]:
df_engagement_data["filename"] = df_engagement_data["filename"].str.replace(".csv","")
df_engagement_data

,time,lp_id,pct_access,engagement_index,filename
0,2020-01-01,93690.0,0.00,NaN,1000
1,2020-01-01,17941.0,0.03,0.90,1000
2,2020-01-01,65358.0,0.03,1.20,1000
3,2020-01-01,98265.0,0.57,37.79,1000
4,2020-01-01,59257.0,0.00,NaN,1000
...,...,...,...,...,...
22324185,2020-12-31,56028.0,0.02,0.20,9927
22324186,2020-12-31,14124.0,0.02,0.99,9927
22324187,2020-12-31,98001.0,0.00,NaN,9927
22324188,2020-12-31,13496.0,0.12,6.51,9927


In [40]:
#fig = make_subplots(rows = 1,cols =2 ,specs = [[{'type':'pie'}, {'type':'pie'}]])
for missing_data in [missing_districts_info,missing_products_info,missing_df_engagement_data]:
    fig = px.pie(missing_data,
                 names= "index", 
                 values = "value",
                 title = "% of Missing values",
                 width=800, height=400,
                color_discrete_sequence=px.colors.sequential.RdBu)

    fig.update_traces(textposition='inside', textinfo='value+label')
    fig.show()

In [8]:
df_engagement_data

,time,lp_id,pct_access,engagement_index,filename
0,2020-01-01,93690.0,0.00,NaN,1000
1,2020-01-01,17941.0,0.03,0.90,1000
2,2020-01-01,65358.0,0.03,1.20,1000
3,2020-01-01,98265.0,0.57,37.79,1000
4,2020-01-01,59257.0,0.00,NaN,1000
...,...,...,...,...,...
22324185,2020-12-31,56028.0,0.02,0.20,9927
22324186,2020-12-31,14124.0,0.02,0.99,9927
22324187,2020-12-31,98001.0,0.00,NaN,9927
22324188,2020-12-31,13496.0,0.12,6.51,9927


In [9]:
df_engagement_data["filename"] = df_engagement_data["filename"].astype(str)
districts_info_df["district_id"] = districts_info_df["district_id"].astype(str)

In [10]:
raw_df_eng_dist = pd.merge(
    df_engagement_data,
    districts_info_df,
    how = "left", 
    left_on = ["filename"],
    right_on = ["district_id"]
)

In [23]:
missing_raw_df_eng_dist = missing_df(raw_df_eng_dist)
missing_raw_df_eng_dist

,index,value
0,time,0.000000
1,lp_id,0.002423
2,pct_access,0.060235
3,engagement_index,24.092292
4,filename,0.000000
5,district_id,0.000000
6,state,21.897529
7,locale,21.897529
8,pct_black/hispanic,21.897529
9,pct_free/reduced,32.847772


In [19]:
print(df_engagement_data["lp_id"].nunique())
print(products_info_df["LP ID"].nunique())

8646
372


In [20]:
raw_df_eng_prod = pd.merge(
    products_info_df, 
    df_engagement_data,
    how      = "left" , 
    left_on  = ["LP ID"] , 
    right_on = ["lp_id"]
)

In [21]:
raw_df_eng_prod_missing = missing_df(raw_df_eng_prod)
raw_df_eng_prod_missing

,index,value
0,LP ID,0.000000
1,URL,0.000000
2,Product Name,0.000000
3,Provider/Company Name,0.000009
4,Sector(s),4.309123
5,Primary Essential Function,4.309123
6,time,0.000026
7,lp_id,0.000026
8,pct_access,0.091715
9,engagement_index,19.478292


In [41]:
#state distribution
fig = px.histogram(districts_info_df, 
                   x = 'state',
                   width=800,
                   height=400).update_xaxes(categoryorder="total descending")
fig.show()

In [45]:
districts_info_df.columns

Index(['district_id', 'state', 'locale', 'pct_black/hispanic',
       'pct_free/reduced', 'county_connections_ratio', 'pp_total_raw'],
      dtype='object')

In [77]:
clean_districts_info_df = districts_info_df[districts_info_df["locale"].isna() == False]

In [110]:
df = pd.DataFrame(clean_districts_info_df["locale"].value_counts()).reset_index()

In [113]:
import plotly.express as px
fig = px.pie(df, 
             values = "locale",
             names = "index",
            color_discrete_sequence= px.colors.sequential.Plasma)
fig.show()